---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [5]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [8]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [10]:
def graph_identification():
    
    # Your Code Here
    G0 = P1_Graphs[0]
#     print('G0: ',max(nx.degree(G0).values()),nx.average_shortest_path_length(G0), nx.average_clustering(G0))
    G1 = P1_Graphs[1]
#     print('G1: ',max(nx.degree(G1).values()),nx.average_shortest_path_length(G1), nx.average_clustering(G1))
    G2 = P1_Graphs[2]
#     print('G2: ',max(nx.degree(G2).values()),nx.average_shortest_path_length(G2), nx.average_clustering(G2))
    G3 = P1_Graphs[3]
#     print('G3: ',max(nx.degree(G3).values()),nx.average_shortest_path_length(G3), nx.average_clustering(G3))
    G4 = P1_Graphs[4]
#     print('G4: ',max(nx.degree(G4).values()),nx.average_shortest_path_length(G4), nx.average_clustering(G4))
    
    return ['PA','SW_L','SW_L','PA','SW_H']
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [12]:
G = nx.read_gpickle('email_prediction.txt')
# print(nx.info(G))

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [13]:
def salary_predictions():
    
    # Your Code Here

    #preliminary work
    
    #import libraries
    from sklearn.naive_bayes import GaussianNB
    from sklearn.linear_model import LogisticRegression
    from sklearn import svm
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score
    from sklearn import metrics
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.model_selection import GridSearchCV
    from sklearn.metrics import classification_report
    from sklearn.model_selection import StratifiedKFold
    import seaborn as sns
#     import matplotlib.pyplot as plt
    from sklearn import preprocessing
    
    
    
    #build the dataframe 
    df = pd.DataFrame(index=G.nodes())
    hits = nx.hits(G)
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['closeness_centrality'] = pd.Series(nx.closeness_centrality(G))
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['node'] = df.index
    df['hub_score'] = df.apply(lambda x:hits[0][x[6]],axis=1)
    df['auth_score'] = df.apply(lambda x:hits[1][x[6]],axis=1)
    #   df = pd.concat([df,pd.get_dummies(df['Department'], prefix='Department')],axis=1)

    
    #perform simple exploratory analysis to understand data better
    
    #Univariate analysis target
    #sns.set(style="darkgrid")
    #ax = sns.countplot(x='ManagementSalary',  data=df.loc[df['ManagementSalary'].notnull()])
    #shows that target feature is not balanced well. 
    #ax = sns.countplot(x='Department',  data=df.loc[df['ManagementSalary'].notnull()])
    #shows that Department 0 by far has the greatest number of ManagementSalary=1 entries. So this will
    #be an important feature in the classifier.
    #plt.show()
    
    #f = plt.figure(figsize=(20,4))
    #f.add_subplot(2,2,1)
    #sns.distplot(df['degree'])
    #f.add_subplot(2,2,2)
    #sns.boxplot(df['degree'])
    #f.add_subplot(2,2,3)
    #sns.distplot(df['clustering'])
    #f.add_subplot(2,2,4)
    #sns.boxplot(df['clustering'])
    #plt.show()
    #shows there are many degree value outliers and some clustering value outliers. There are many nodes
    #with low degree values and very few nodes with very high degree values. The clustering coefficient is centered
    #around a value of about 0.35. 

    
    #Simply binary classifier
    clf = LogisticRegression(class_weight='balanced', random_state=0, C=10, penalty='l2')
    
    #Split data into testing and training
    df_X_test =  df.loc[df['ManagementSalary'].isnull()].drop(['ManagementSalary','Department','node'], axis=1)
    df_X_train = df.loc[df['ManagementSalary'].notnull()].drop(['ManagementSalary','Department','node'], axis=1)
    df_y_train  = df.loc[df['ManagementSalary'].notnull()]['ManagementSalary']   
    
    X_train, X_test, y_train, y_test = train_test_split(df_X_train, df_y_train, random_state=0)

    #fit the model
    clf.fit(df_X_train,df_y_train.values.ravel())

    #evaluate model
#     fpr, tpr, _ = metrics.roc_curve(y_test, clf.decision_function(X_test))
#     print(metrics.auc(fpr, tpr))
    
    y_pred = [item[1] for item in clf.predict_proba(df_X_test)]
    nodes = df.loc[df['ManagementSalary'].isnull()]['node'].tolist()

    d = {'Node_ID':nodes,'Probability':y_pred}
    df_prob = pd.DataFrame(d).set_index('Node_ID')
#     print(df_prob.head())
    
    

    
#     grid_values = {'penalty': ['l1','l2'], 'C': [0.001,0.01,0.1,1,10,100,1000]}
#     grid = GridSearchCV(clf, param_grid=grid_values, scoring='roc_auc')
#     grid_result = grid.fit(X_train,y_train.values.ravel())
#     print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


#     # Number of trees in random forest
#     n_estimators = [int(x) for x in np.linspace(start=1, stop=1000, num=20)]
#     # Number of features to consider at every split
#     max_features = ['auto', 'None']
#     # Maximum number of levels in tree
#     max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
#     max_depth.append(None)
#     # Minimum number of samples required to split a node
#     min_samples_split = [2, 5, 10]
#     # Minimum number of samples required at each leaf node
#     min_samples_leaf = [1, 2, 4]
#     # Method of selecting samples for training each tree
#     bootstrap = [True,False]  # Create the random grid
#     random_grid = {'n_estimators': n_estimators,
#                    'max_features': max_features,
#                    'max_depth': max_depth,
#                    'min_samples_split': min_samples_split,
#                    'min_samples_leaf': min_samples_leaf,
#                    'bootstrap': bootstrap}

#     # Use the random grid to search for best hyperparameters
#     # First create the base model to tune
#     rf = RandomForestClassifier()
#     # Random search of parameters, using 3 fold cross validation,
#     # search across 100 different combinations, and use all available cores
#     rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=5, cv=5, verbose=2,
#                                    random_state=42, n_jobs=-1, scoring='roc_auc')  # Fit the random search model
#     X_train, X_test, y_train, y_test = train_test_split(df_X_train, df_y_train, random_state=0)
#     rf_random.fit(X_train, y_train.values.ravel())
#     print(rf_random.best_params_)
    

    
    
    return df_prob
salary_predictions()

AttributeError: 'Graph' object has no attribute '_node'

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [50]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [33]:
def new_connections_predictions():
    
    # Your Code Here
    from sklearn.naive_bayes import GaussianNB
    from sklearn.linear_model import LogisticRegression
    from sklearn import svm
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score
    from sklearn import metrics
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.model_selection import GridSearchCV
    from sklearn.metrics import classification_report
    from sklearn.model_selection import StratifiedKFold
    import seaborn as sns
#     import matplotlib.pyplot as plt
    from sklearn import preprocessing
    
    
    future_connections['preferential_attachment'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
    future_connections['common_neighbors'] = future_connections.index.map(lambda person: len(list(nx.common_neighbors(G, person[0], person[1]))))
    
    train = future_connections.loc[future_connections['Future Connection'].notnull()]
    pred = future_connections.loc[future_connections['Future Connection'].isnull()]

#     print(train.head())
    
    #Simply binary classifier
    clf = LogisticRegression(class_weight='balanced', random_state=0, C=10, penalty='l2')
    
    #Split data into testing and training
    df_X_test = preprocessing.scale(future_connections.loc[future_connections['Future Connection'].isnull()].drop(['Future Connection'], axis=1))
    df_X_train = future_connections.loc[future_connections['Future Connection'].notnull()].drop(['Future Connection'], axis=1)
    df_y_train  = future_connections.loc[future_connections['Future Connection'].notnull()]['Future Connection']   
    
    X_train, X_test, y_train, y_test = train_test_split(preprocessing.scale(df_X_train), df_y_train, random_state=0)

    #fit the model
    clf.fit(X_train,y_train.values.ravel())

    #evaluate model
    fpr, tpr, _ = metrics.roc_curve(y_train, clf.decision_function(X_train))
#     print(metrics.auc(fpr, tpr))
    
    y_pred = [item[1] for item in clf.predict_proba(df_X_test)]
    edges = future_connections.loc[future_connections['Future Connection'].isnull()].index.values

    d = {'Edges':edges,'Probability':y_pred}
    df_prob = pd.DataFrame(d).set_index('Edges')

    
    return df_prob
# new_connections_predictions()